In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [4]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [5]:
# df.columns = df.columns.str.lower().str.replace(' ', '_')

# categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

# for c in categorical_columns:
#     df[c] = df[c].str.lower().str.replace(' ', '_')
   



In [6]:
# df.head().T

In [7]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [8]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [9]:
fill_categorical = df.select_dtypes(include=['object']).columns
fill_numerical = df.select_dtypes(include=['number']).columns

df[fill_categorical] = df[fill_categorical].fillna('NA')
df[fill_numerical] = df[fill_numerical].fillna(0.0)
df.head()


,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [10]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [13]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [14]:
len(df)

1462

In [15]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [16]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']


In [17]:
df_full_train = df_full_train.reset_index(drop=True)
df_full_train.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [19]:
df_full_train.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,social_media,manufacturing,2,44403.0,self_employed,australia,1,0.71,0
1,events,retail,3,38048.0,student,north_america,6,0.97,1
2,social_media,education,2,71399.0,NA,europe,1,0.51,1
3,referral,education,2,47912.0,employed,australia,1,0.04,0
4,paid_ads,healthcare,1,34806.0,employed,europe,4,0.32,1


In [20]:
df_full_train.converted.value_counts(normalize=True)

converted
1    0.607357
0    0.392643
Name: proportion, dtype: float64

In [21]:
global_converted = df_full_train.converted.mean()

In [22]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count','lead_score']

In [23]:
categorical = [
    'lead_source',
    'industry',
    'employment_status',
    'location',

]

In [24]:
df_full_train[categorical].nunique()

lead_source          6
industry             8
employment_status    5
location             8
dtype: int64

In [25]:
from IPython.display import display

In [26]:
for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c).converted.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_converted
    df_group['risk'] = df_group['mean'] / global_converted
    display(df_group)
    print()
    print()

lead_source


,mean,count,diff,risk
lead_source,,,,
NA,0.669725,109,0.062368,1.102688
events,0.592965,199,-0.014392,0.976304
organic_search,0.615721,229,0.008364,1.013771
paid_ads,0.428571,210,-0.178785,0.705634
referral,0.786408,206,0.179051,1.294804
social_media,0.583333,216,-0.024023,0.960446




industry


,mean,count,diff,risk
industry,,,,
NA,0.557522,113,-0.049835,0.917948
education,0.790210,143,0.182853,1.301064
finance,0.556886,167,-0.050470,0.916901
healthcare,0.593333,150,-0.014023,0.976911
manufacturing,0.621429,140,0.014072,1.023169
other,0.612903,155,0.005547,1.009132
retail,0.566265,166,-0.041092,0.932343
technology,0.562963,135,-0.044394,0.926907




employment_status


,mean,count,diff,risk
employment_status,,,,
NA,0.571429,84,-0.035928,0.940845
employed,0.671937,253,0.064580,1.106330
self_employed,0.635739,291,0.028382,1.046731
student,0.659722,288,0.052366,1.086219
unemployed,0.462451,253,-0.144906,0.761415




location


,mean,count,diff,risk
location,,,,
NA,0.680000,50,0.072643,1.119606
africa,0.592857,140,-0.014500,0.976127
asia,0.588608,158,-0.018749,0.969130
australia,0.612903,155,0.005547,1.009132
europe,0.641176,170,0.033820,1.055684
middle_east,0.644172,163,0.036815,1.060615
north_america,0.589888,178,-0.017469,0.971238
south_america,0.554839,155,-0.052518,0.913530


In [27]:
from sklearn.metrics import mutual_info_score

In [28]:
def mutual_info_converted_score(series):
    return mutual_info_score(series, df_full_train.converted)

In [29]:
mi = df_full_train[categorical].apply(mutual_info_converted_score)
mi.sort_values(ascending=False)


lead_source          0.025665
employment_status    0.013258
industry             0.011685
location             0.002253
dtype: float64

In [30]:
df_full_train[numerical].corrwith(df_full_train.converted).abs()

number_of_courses_viewed    0.442068
annual_income               0.029612
interaction_count           0.378482
lead_score                  0.225641
dtype: float64

In [31]:
corr_matrix = df[numerical].corr()

In [32]:
pair_correlations = {
    'interaction_count and lead_score': corr_matrix.loc['interaction_count', 'lead_score'],
    'number_of_courses_viewed and lead_score': corr_matrix.loc['number_of_courses_viewed', 'lead_score'],
    'number_of_courses_viewed and interaction_count': corr_matrix.loc['number_of_courses_viewed', 'interaction_count'],
    'annual_income and interaction_count': corr_matrix.loc['annual_income', 'interaction_count'],
}

In [33]:
biggest_corr_pair = max(pair_correlations)
biggest_corr_pair

'number_of_courses_viewed and lead_score'

In [34]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [37]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [38]:
model.intercept_[0]

np.float64(-0.0691472802783609)

In [39]:
model.coef_[0].round(2)

array([-0.  , -0.01,  0.03,  0.  ,  0.01, -0.1 , -0.02,  0.05, -0.02,
       -0.01, -0.  , -0.01, -0.03, -0.02,  0.31,  0.05,  0.02, -0.01,
       -0.01, -0.12,  0.08, -0.03,  0.  , -0.01, -0.01, -0.01,  0.01,
        0.01, -0.03, -0.03,  0.45])

In [40]:
model.predict_proba(X_train)

array([[0.42085657, 0.57914343],
       [0.12716509, 0.87283491],
       [0.41183893, 0.58816107],
       ...,
       [0.25265784, 0.74734216],
       [0.3302157 , 0.6697843 ],
       [0.14407823, 0.85592177]], shape=(876, 2))

In [41]:
y_pred = model.predict_proba(X_val)[:, 1]
converted_decision = (y_pred >= 0.5)
(y_val == converted_decision).mean()
acc = accuracy_score(y_val, converted_decision)
acc

0.6996587030716723

In [43]:
dicts_full_train = df_full_train[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)

In [44]:
y_full_train = df_full_train.converted.values
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_full_train, y_full_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [45]:
dicts_test = df_test[categorical + numerical].to_dict(orient='records')
X_test = dv.transform(dicts_test)
y_pred = model.predict_proba(X_test)[:, 1]
converted_decision = (y_pred >= 0.5)
(converted_decision == y_test).mean()

np.float64(0.7372013651877133)

In [50]:
full_features = categorical + numerical 


dv_train_full = df_train[full_features].to_dict(orient='records')
dv_val_full = df_val[full_features].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X_train_full = dv.fit_transform(dv_train_full)
X_val_full = dv.transform(dv_val_full)


model_full = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model_full.fit(X_train_full, y_train)
y_pred_full = model_full.predict(X_val_full)
acc = accuracy_score(y_val, y_pred_full) 
print(f"Baseline Accuracy (acc): {acc:.6f}\n---")

features_to_check = ['industry', 'employment_status', 'lead_score']

for feature in features_to_check:
   
    features_to_keep = [f for f in full_features if f != feature]
    
    dv_train_subset = df_train[features_to_keep].to_dict(orient='records')
    dv_val_subset = df_val[features_to_keep].to_dict(orient='records')
    
   
    dv_subset = DictVectorizer(sparse=False)
    X_train_new = dv_subset.fit_transform(dv_train_subset)
    X_val_new = dv_subset.transform(dv_val_subset)
    

    model = LogisticRegression(solver='liblinear', random_state=42)
    model.fit(X_train_new, y_train)
    
    y_pred = model.predict(X_val_new)
    acc_new = accuracy_score(y_val, y_pred)
    
    diff = acc - acc_new
    
    print(f"Feature excluded: {feature:<17}  New Accuracy: {acc_new:.6f}  Difference: {diff:.6f}")

Baseline Accuracy (acc): 0.699659
---
Feature excluded: industry           New Accuracy: 0.699659  Difference: 0.000000
Feature excluded: employment_status  New Accuracy: 0.696246  Difference: 0.003413
Feature excluded: lead_score         New Accuracy: 0.706485  Difference: -0.006826


In [54]:

full_features = categorical + numerical 
dv_train_full = df_train[full_features].to_dict(orient='records')
dv_val_full = df_val[full_features].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X_train_full = dv.fit_transform(dv_train_full)
X_val_full = dv.transform(dv_val_full)


Cs = [0.01, 0.1, 1, 10, 100]
accuracies = {}


for C in Cs:
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train_full, y_train)
    
    y_pred = model.predict(X_val_full)
    acc = accuracy_score(y_val, y_pred)
    accuracies[C] = round(acc, 3)

for C, acc in accuracies.items():
    print(f"C={C:<5}: Accuracy = {acc:.3f}")

C=0.01 : Accuracy = 0.700
C=0.1  : Accuracy = 0.700
C=1    : Accuracy = 0.700
C=10   : Accuracy = 0.700
C=100  : Accuracy = 0.700


In [55]:
best_C = max(accuracies, key=accuracies.get)
best_C

0.01